In [1]:
#tqdmfrom nipype.interfaces import niftyreg
import numpy as np
from skimage import io
from pathlib import Path
import re
import ants
from skimage.transform import resize
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

In [2]:
directory_seg = Path('james_preds')
files_seg  = list(directory_seg.glob('*_seg.npy'))
files_seg = sorted([x.as_posix() for x in files_seg])

for i in tqdm(range(len(files_seg))):
    tmp = np.load(files_seg[i])
    tmp = np.reshape(tmp,(1,507,507,252))
    io.imsave(re.sub('.npy','.tif',files_seg[0]),tmp)

In [3]:
files_seg  = list(directory_seg.glob('*_seg.tif'))
files_seg = sorted([x.as_posix() for x in files_seg])
files_seg[0]

'james_preds/XYZres006_0001_seg.tif'

In [4]:
image_path = Path('../TBI')
images = list(image_path.glob('vbm*/*?[0-9].tif'))
images = sorted([x.as_posix() for x in images])
len(images)

94

for i in tqdm(range(len(images))):
    image = io.imread(images[i])
    image = np.swapaxes(image,0,1)
    image = np.swapaxes(image,1,3)
    image = resize(image,(2,507,507,252),preserve_range=True)
    zeros = np.zeros((1,507,507,252)).astype('uint16')
    image = np.append(image,zeros,axis = 0)
    image = image.astype('float16')
    io.imsave('james_preds/'+re.sub('.tif','_upsamp.tif',Path(images[i]).name),image)

In [5]:
#'james_preds/'+re.sub('.tif','_upsamp.tif',Path(images[i]).name

In [6]:
images = list(directory_seg.glob('*_0001_upsamp.tif'))
images = sorted([x.as_posix() for x in images])
images[0]

'james_preds/XYZres006_0001_upsamp.tif'

In [7]:
for i in tqdm(range(len(images))):
    fix = ants.image_read(re.sub('_0001','',images[i]))
    mov = ants.image_read(images[0])
    fix_mask = ants.image_read(re.sub('upsamp','seg',re.sub('_0001','',images[i])))
    mov_mask = ants.image_read(re.sub('upsamp','seg',images[i]))
    mytx = ants.registration(fixed = fix,
                            moving = mov,
                            type_of_transform = 'Rigid'
                            )
    warpedmask = ants.apply_transforms(fixed = fix_mask,
                                       moving = mov_mask,
                                       transformlist = mytx['fwdtransforms'],
                                       interpolator = 'nearestNeighbor'
                                      )
    io.imsave(re.sub('seg','seg_qwarped',re.sub('upsamp','seg',images[i])),warpedmask.numpy())
    io.imsave(re.sub('seg','seg_qwarped',re.sub('upsamp','seg',re.sub('_0001','',images[i]))),fix_mask.numpy())
    #print(2 * np.sum(warpedmask.numpy()*fix_mask.numpy())/(np.sum(warpedmask.numpy())+np.sum(fix_mask.numpy())))

  0%|          | 0/47 [00:05<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
fix_mask.numpy()

In [8]:
PRE = []
POST = []
for i in tqdm(range(len(images))):
    fix = io.imread(re.sub('_0001','',images[i]))
    mov = io.imread(images[i])
    PRE.append(mean_squared_error(fix.flatten(),mov.flatten()))
    mov_2 = io.imread(re.sub('seg','seg_warped',re.sub('upsamp','seg',images[i])))
    fix_2 = io.imread(re.sub('seg','seg_warped',re.sub('upsamp','seg',re.sub('_0001','',images[i]))))
    POST.append(mean_squared_error(fix_2.flatten(),mov_2.flatten()))

100%|██████████| 47/47 [02:29<00:00,  3.19s/it]


In [9]:
PRE

[5953.416,
 1617.9143,
 1048.1672,
 1502.8165,
 794.3065,
 1489.714,
 982.5049,
 1335.8333,
 795.2718,
 1368.7443,
 1250.2186,
 1354.9563,
 1215.2213,
 1274.0315,
 998.3873,
 1236.9438,
 754.36755,
 947.00305,
 941.0864,
 901.8656,
 688.85944,
 1578.668,
 1178.5963,
 1674.8424,
 1457.8485,
 1766.109,
 1142.2817,
 1382.1937,
 974.9622,
 1376.7922,
 1143.0839,
 1337.4873,
 973.6096,
 1223.294,
 1253.3029,
 1140.7921,
 1440.4901,
 1046.4585,
 1054.0519,
 665.7226,
 1821.5646,
 958.90955,
 1805.2811,
 976.31683,
 1729.9274,
 915.14026,
 940.67035]

In [10]:
POST

[0.023449084,
 0.008380991,
 0.006840676,
 0.010978405,
 0.006251433,
 0.00970842,
 0.006983274,
 0.0075406227,
 0.004930349,
 0.0073936554,
 0.010658566,
 0.0067826454,
 0.008855408,
 0.0055657197,
 0.007908473,
 0.012030194,
 0.0057086116,
 0.0089822905,
 0.0071047847,
 0.007641956,
 0.0046119303,
 0.008855655,
 0.0076116207,
 0.011802502,
 0.009816253,
 0.012252759,
 0.0058174473,
 0.009166571,
 0.006871613,
 0.009065392,
 0.008118612,
 0.009902241,
 0.006838993,
 0.0090700695,
 0.00959523,
 0.007883865,
 0.010389486,
 0.0075636404,
 0.0076926225,
 0.0058117816,
 0.010067517,
 0.0057160677,
 0.009636866,
 0.006226671,
 0.008801947,
 0.00511097,
 0.0062081455]

In [11]:
print(np.mean(PRE))
print(np.std(PRE))

1306.5964
747.25244


In [12]:
print(np.mean(POST))
print(np.std(POST))

0.008387277
0.0029221098
